### SparkNLP Word Analaysis POS Lemma

In [ ]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.ml import PipelineModel
import sparknlp
import sys
import time
import pandas as pd
import numpy as np
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *
from sparknlp.pretrained import ResourceDownloader
from pyspark.sql.types import StringType, IntegerType
from pathlib import Path
if sys.version_info[0] < 3:
    from urllib import urlretrieve
else:
    from urllib.request import urlretrieve

In [ ]:
spark = sparknlp.start()

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

#### POS

In [ ]:
data = spark.read.text('text_all_word.txt').toDF('text')

In [ ]:
data.show(5)

In [ ]:
# Default POS Setting
# document = DocumentAssembler().setInputCol('text').setOutputCol('document') # text data input columns
# tokenizer = Tokenizer().setInputCols('document').setOutputCol('token')
# pos = PerceptronModel.pretrained().setInputCols('document', 'token').setOutputCol('pos')
# pipeline = Pipeline().setStages([document, tokenizer, pos])

In [ ]:
# Turkish POS model (name='pos_ud_imst', lang='tr') from (https://nlp.johnsnowlabs.com/docs/en/models) 
document = DocumentAssembler().setInputCol('text').setOutputCol('document') # text data input columns
tokenizer = Tokenizer().setInputCols('document').setOutputCol('token')
pos = PerceptronModel.pretrained(name='pos_ud_imst', lang='tr').setInputCols('document', 'token').setOutputCol('pos')
pipeline = Pipeline().setStages([document, tokenizer, pos])

In [ ]:
model = pipeline.fit(data)

In [ ]:
result = model.transform(data)

In [ ]:
result.show(5)

In [ ]:
result.printSchema()

In [ ]:
# stored = result\
#   .select('text', 'pos.begin', 'pos.end', 'pos.result', 'pos.metadata')\
#   .toDF('text', 'pos_begin', 'pos_end', 'pos_result', 'pos_meta')\
#   .cache()

In [ ]:
stored = result\
  .select('text', 'pos.result')\
  .toDF('text', 'pos_result')\
  .cache()

In [ ]:
stored.printSchema()

In [ ]:
df = stored.toPandas()
df

#### Lemmatizer

In [ ]:
# Original Code
# import time
# 
# documentAssembler = DocumentAssembler() \
#     .setInputCol("sentence") \
#     .setOutputCol("document")
# 
# tokenizer = Tokenizer() \
#     .setInputCols(["document"]) \
#     .setOutputCol("token")
# 
# lemmatizer = LemmatizerModel.pretrained() \ 
#     .setInputCols(["token"]) \
#     .setOutputCol("lemma")
# 
# spell = NorvigSweetingModel.pretrained() \
#     .setInputCols(["token"]) \
#     .setOutputCol("spell")
# 
# embeddings = WordEmbeddingsModel.pretrained() \
#     .setInputCols(["document", "token"]) \
# 
# ner_dl = NerDLModel().pretrained() \
#     .setInputCols(["document", "token", "embeddings"]) \
#     .setOutputCol("ner_dl")
# 
# finisher = Finisher() \
#     .setInputCols(["ner_dl", "lemma", "spell"]) \
#     .setIncludeMetadata(True)
# 
# pipeline_fast_dl = Pipeline(stages = [
#     documentAssembler, 
#     tokenizer, 
#     lemmatizer, 
#     spell, 
#     embeddings, 
#     ner_dl, 
#     finisher])

In [ ]:
documentAssembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

lemmatizer = LemmatizerModel.pretrained(name='lemma', lang='tr') \
    .setInputCols(["token"]) \
    .setOutputCol("lemma")

# spell = NorvigSweetingModel.pretrained() \
#     .setInputCols(["token"]) \
#     .setOutputCol("spell")
# 
# embeddings = WordEmbeddingsModel.pretrained() \
#     .setInputCols(["document", "token"]) \
# 
# ner_dl = NerDLModel().pretrained() \
#     .setInputCols(["document", "token", "embeddings"]) \
#     .setOutputCol("ner_dl")

finisher = Finisher() \
    .setInputCols(["lemma"]) \
    .setIncludeMetadata(False)

pipeline_fast_dl = Pipeline(stages = [
    documentAssembler, 
    tokenizer, 
    lemmatizer,  
    finisher])